In [2]:
from azure.ai.ml import MLClient, command, Input
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)


## Create connection to Azure ML service

In [3]:
default_azure_credential = DefaultAzureCredential()

ml_client = MLClient(
    credential=default_azure_credential,
    subscription_id="YOUR SUBSCRIPTION",
    resource_group_name="YOUR RESOURCE GROUP",
    workspace_name="YOUR WORKSPACE NAME",
)


## Register model (use existing score.py in the app python module

In [19]:
file_model = Model(
    path="../app/",
    type="custom_model",
    name="Fit",
    description="Fit app module"
)
ml_client.models.create_or_update(file_model)

Model({'job_name': None, 'is_anonymous': False, 'auto_increment_version': False, 'name': 'Fit', 'description': 'Fit app module', 'tags': {}, 'properties': {}, 'id': '/subscriptions/54108101-44be-4e65-a655-ea14694b0a64/resourceGroups/test2-rg/providers/Microsoft.MachineLearningServices/workspaces/versioning/models/Fit/versions/1', 'base_path': './', 'creation_context': <azure.ai.ml._restclient.v2022_05_01.models._models_py3.SystemData object at 0x7f35fc476f80>, 'serialize': <msrest.serialization.Serializer object at 0x7f36089d3f70>, 'version': '1', 'latest_version': None, 'path': 'azureml://subscriptions/54108101-44be-4e65-a655-ea14694b0a64/resourceGroups/test2-rg/workspaces/versioning/datastores/workspaceblobstore/paths/LocalUpload/5b6d0714cd2c84103fd5241824947d2b/app', 'utc_time_created': None, 'flavors': None, 'arm_type': 'model_version', 'type': 'custom_model'})

## Deploy the model as REST endpoint

In [27]:
# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name="Fit",
    description="Versioned python endpoint for FIT",
    auth_mode="key",
    tags={
        "function": "fit",
        "model_type": "sklearn.LinearRegression",
    },
)

endpoint = ml_client.begin_create_or_update(endpoint)
print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

#Create also local endpoint
local_endpoint = ManagedOnlineEndpoint(
    name="fit-local", description="local endpoint"
)

ml_client.online_endpoints.begin_create_or_update(local_endpoint, local=True)



Updating local endpoint (fit-local) .

Endpoint fit provisioning state: Succeeded


Done (0m 5s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': 'Failed', 'scoring_uri': None, 'swagger_uri': None, 'name': 'fit-local', 'description': 'local endpoint', 'tags': {}, 'properties': {}, 'id': None, 'base_path': './', 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x7ff03c228fd0>, 'auth_mode': 'key', 'location': 'local', 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [32]:
#Test local deploy first

#Use local files as model on local deployment
local_model = Model(path="../app/")

env = Environment(
    conda_file="../conda.yml",
    image="mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:20220516.v3",
)

deployment = ManagedOnlineDeployment(
    name="local-v100",
    endpoint_name=local_endpoint.name,
    model=local_model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../.", scoring_script="score.py"
    )
)

#Test local deployment first

deployment = ml_client.begin_create_or_update(deployment, local=True)

Updating local deployment (fit-local / local-v100) 
Building Docker image from Dockerfile.
Step 1/6 : FROM mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:20220516.v3
 ---> 2ba524631f05
Step 2/6 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> cd9d8004f482
Step 3/6 : WORKDIR /var/azureml-app/
 ---> Using cache
 ---> 9833df5c499b
Step 4/6 : COPY conda.yml /var/azureml-app/
 ---> Using cache
 ---> 411cf2c805fe
Step 5/6 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Using cache
 ---> a8e14e67f1d3
Step 6/6 : CMD ["conda", "run", "--no-capture-output", "-n", "inf-conda-env", "runsvdir", "/var/runit"]
 ---> Using cache
 ---> dcd99470c340
Successfully built dcd99470c340
Successfully tagged fit-local:local-v100

Starting up endpoint...Done (0m 20s)


In [33]:
#Verify local endpoints
ml_client.online_endpoints.get(name="fit-local", local=True)
ml_client.online_deployments.get_logs(
    name="local-v100", endpoint_name="fit-local", local=True, lines=50
)

"2022-08-23T11:34:02,566123961+00:00 | gunicorn/run | ###############################################\r\n2022-08-23T11:34:02,567674790+00:00 | gunicorn/run | Dynamic Python Package Installation\r\n2022-08-23T11:34:02,569201718+00:00 | gunicorn/run | ###############################################\r\n2022-08-23T11:34:02,571163054+00:00 | gunicorn/run | \r\n2022-08-23T11:34:02,573051289+00:00 | gunicorn/run | Dynamic Python package installation is disabled.\r\n2022-08-23T11:34:02,575217729+00:00 | gunicorn/run | \r\n2022-08-23T11:34:02,577143964+00:00 | gunicorn/run | ###############################################\r\n2022-08-23T11:34:02,578974598+00:00 | gunicorn/run | AzureML Inference Server\r\n2022-08-23T11:34:02,581206439+00:00 | gunicorn/run | ###############################################\r\n2022-08-23T11:34:02,582738367+00:00 | gunicorn/run | \r\n2022-08-23T11:34:02,613363932+00:00 | gunicorn/run | Starting AzureML Inference Server HTTP.\r\n\r\nAzure ML Inferencing HTTP server v

In [34]:
#Deploy to Azure

#Get the latest version of FIT from model repository
latest_model_version = max(
    [int(m.version) for m in ml_client.models.list(name="Fit")]
)
model = ml_client.models.get(name="Fit", version=latest_model_version)

deployment = ManagedOnlineDeployment(
    name="v100",
    endpoint_name=endpoint.name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../.", scoring_script="score.py"
    ),
    instance_type="Standard_DS1_v2",
    instance_count=1,
)
deployment = ml_client.begin_create_or_update(deployment)

Check: endpoint fit exists
Uploading python-rest-versioning (0.13 MBs): 100%|██████████| 134042/134042 [00:00<00:00, 269450.43it/s]


Creating/updating online deployment v100 

..........................................................................................................................................

Done (12m 23s)


## Operate endpoint

In [36]:
ml_client.online_deployments.get_logs(
    name="v100", endpoint_name="fit",  lines=100
)

'Instance status:\nSystemSetup: Succeeded\nUserContainerImagePull: Succeeded\nModelDownload: Succeeded\nUserContainerStart: Succeeded\n\nContainer events:\nKind: Pod, Name: Downloading, Type: Normal, Time: 2022-08-23T12:03:52.586592Z, Message: Start downloading models\nKind: Pod, Name: Pulling, Type: Normal, Time: 2022-08-23T12:03:52.760815Z, Message: Start pulling container image\nKind: Pod, Name: Pulled, Type: Normal, Time: 2022-08-23T12:04:36.592985Z, Message: Container image is pulled successfully\nKind: Pod, Name: Downloaded, Type: Normal, Time: 2022-08-23T12:04:36.592985Z, Message: Models are downloaded successfully\nKind: Pod, Name: Created, Type: Normal, Time: 2022-08-23T12:04:36.85377Z, Message: Created container inference-server\nKind: Pod, Name: Started, Type: Normal, Time: 2022-08-23T12:04:37.119539Z, Message: Started container inference-server\nKind: Pod, Name: ContainerReady, Type: Normal, Time: 2022-08-23T12:04:51.355475525Z, Message: Container is ready\n\nContainer logs